In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import make_scorer
from sklearn.svm import SVC
import pandas as pd

In [ ]:
df = pd.read_csv('./agaricus_lepiota_small_c.csv')

# Codificação do atributo de saída (class): e → 0 e p → 1

In [ ]:
transformers = [
   ('oe_class', OrdinalEncoder(categories=[['e', 'p']]), ['class']),
]

ct = ColumnTransformer(transformers=transformers, remainder='passthrough')

y_oe = ct.fit_transform(df)

df = pd.DataFrame(y_oe, columns=df.columns)
df['class'] = df['class'].astype(int)

df.head()

# Exclusão de dados faltantes

In [ ]:
# atributos_categoricos = ['stalk-root']

# transformers = [
#     ('imp_cat', SimpleImputer(strategy='constant', fill_value='N'), atributos_categoricos)
# ]

# ct_imp = ColumnTransformer(
#     transformers, remainder='drop'
# )

# X_imp_vals = ct_imp.fit_transform(df)
# X_imp_vals = X_imp_vals[:, 0]  # Selecionar apenas a primeira coluna (stalk-root)

# df['stalk-root'] = X_imp_vals
# df['stalk-root']

df = df.drop('stalk-root', axis=1) 

df
# print(ct_imp.transformers_[0][1].statistics_)

# Codificação de atributos categóricos

In [ ]:
nominal_attr = ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 
           'gill-color', 'stalk-shape', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 
           'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 
           'habitat']

for column in nominal_attr:
    transformers = [
        ('oe_' + column, OneHotEncoder(), [column])
    ]

    ct_oe = ColumnTransformer(
        transformers, remainder='passthrough'
    )

    X_oe = ct_oe.fit_transform(df)

    df[column] = X_oe[:, 0]

df 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, make_scorer
import numpy as np

In [ ]:
def custom_positive_accuracy(y_true, y_pred):
    # positive_samples = (y_true == 1)
    return accuracy_score(y_true[y_true == 1], y_pred[y_true == 1])

# positive_accuracy_scorer = make_scorer(custom_positive_accuracy)

In [ ]:
X = df.drop('class', axis=1)
y = df['class']

In [ ]:
from sklearn.model_selection import GridSearchCV #busca exaustiva para otimização de hiperparâmetro por validação cruzada

# Validação cruzada em dois níveis com KNN

In [ ]:
def do_knn():
    positive_accuracy_scorer = make_scorer(custom_positive_accuracy)

    k1 = 10 #controla o número de vias da validação cruzada para estimar o desempenho do modelo
    k2 = 5 #controla o número de vida da validação cruzada para otimização de hiperparametros

    #usar o protocolo de validação cruzada estratificada
    skf = StratifiedKFold(n_splits=k1, shuffle=True, random_state=1)

    acuracias = []

    #a função split retorna os índices das instâncias que devem ser usadas para o treinamento e o teste.
    for idx_treino, idx_teste in skf.split(X, y):
        
        #extrair as instâncias de treinamento de acordo com os índices fornecidos pelo skf.split
        X_treino = X.iloc[idx_treino]
        y_treino = y.iloc[idx_treino]
        
        #extrair as instâncias de teste de acordo com os índices fornecidos pelo skf.split
        X_teste = X.iloc[idx_teste]
        y_teste = y.iloc[idx_teste]
        
        #colocar todas as variáveis na mesma escala, usando o conjunto de treinamento para calcular os parâmetros da escala
        ss = StandardScaler()
        ss.fit(X_treino)
        X_treino = ss.transform(X_treino)
        X_teste = ss.transform(X_teste)
        
        #combinações de parametros otimizar. Aqui estamos apenas otimizando o número de vizinhos mais próximos para o knn (k).
        #Entretanto, podemos colocar todos os valores de todos os parametros. O sklearn se encarrega de gerar todas as combinações.
        params = {'n_neighbors' : range(1,30,2)}

        #instanciar um KNN com parametros padrão
        knn = KNeighborsClassifier()

        #instanciar um GridSearchCV com k2 vias.
        knn = GridSearchCV(knn, params, cv=StratifiedKFold(n_splits=k2), scoring=positive_accuracy_scorer)
        
        #realizar a otimização dos hiperparâmetros e treinar o modelo final com a melhor combinação de hiperparametros com todos os dados de treinamento
        knn.fit(X_treino, y_treino)
        
        #calcular a acurácia no conjunto de testes desta iteração e salvar na lista.
        # acuracias.append(accuracy_score(y_teste, knn.predict(X_teste)))
        acuracias.append(positive_accuracy_scorer(knn, X_teste, y_teste))  # Use a métrica personalizada para calcular a acurácia
        # acuracias.append(y_teste, knn.predict(X_teste))
        # acuracias.append(positive_accuracy_scorer(y_teste, knn.predict(X_teste)))  # Use a métrica personalizada para calcular a acurácia
        # acuracias.append(knn.best_score_)  # Use a métrica personalizada para calcular a acurácia da classe positiva
        
    #calcular as estatísticas da validação cruzada. Estas estatísticas nos dão uma confiança que, na média, este é o desempenho esperado
    #do classificador no mundo real.
    # print("min: %.2f, max: %.2f, avg +- std: %.2f+-%.2f" % (min(acuracias), max(acuracias), np.mean(acuracias), np.std(acuracias)))
    return acuracias

# Validação cruzada em dois níveis com SVM

In [ ]:
def do_svm():
    k1 = 10 #controla o número de vias da validação cruzada para estimar o desempenho do modelo
    k2 = 5 #controla o número de vida da validação cruzada para otimização de hiperparametros

    positive_accuracy_scorer = make_scorer(custom_positive_accuracy)

    #usar o protocolo de validação cruzada estratificada
    skf = StratifiedKFold(n_splits=k1, shuffle=True, random_state=1)

    acuracias = []

    #a função split retorna os índices das instâncias que devem ser usadas para o treinamento e o teste.
    for idx_treino, idx_teste in skf.split(X, y):
        
        #extrair as instâncias de treinamento de acordo com os índices fornecidos pelo skf.split
        X_treino = X.iloc[idx_treino]
        y_treino = y.iloc[idx_treino]
        
        #extrair as instâncias de teste de acordo com os índices fornecidos pelo skf.split
        X_teste = X.iloc[idx_teste]
        y_teste = y.iloc[idx_teste]
        
        #colocar todas as variáveis na mesma escala, usando o conjunto de treinamento para calcular os parâmetros da escala
        ss = StandardScaler()
        ss.fit(X_treino)
        X_treino = ss.transform(X_treino)
        X_teste = ss.transform(X_teste)
        
        #combinações de parametros otimizar. Aqui estamos apenas otimizando o número de vizinhos mais próximos para o knn (k).
        #Entretanto, podemos colocar todos os valores de todos os parametros. O sklearn se encarrega de gerar todas as combinações.
        params = {
            'C': [0.1, 1, 10, 100, 1000],
            'gamma': ['scale', 'auto', 2e-2, 2e-3, 2e-4],
        }
        #instanciar um SVM com parametros padrão
        svm = SVC(kernel='rbf')

        #instanciar um GridSearchCV com k2 vias.
        svm = GridSearchCV(svm, params, cv=StratifiedKFold(n_splits=k2), scoring=positive_accuracy_scorer)

        #realizar a otimização dos hiperparâmetros e treinar o modelo final com a melhor combinação de hiperparametros com todos os dados de treinamento
        svm.fit(X_treino, y_treino)
        
        #calcular a acurácia no conjunto de testes desta iteração e salvar na lista.
        # acuracias.append(accuracy_score(y_teste, svm.predict(X_teste)))
        acuracias.append(positive_accuracy_scorer(svm, X_teste, y_teste))  # Use a métrica personalizada para calcular a acurácia
        # acuracias.append(svm.best_score_)  # Use a métrica personalizada para calcular a acurácia da classe positiva
        
    #calcular as estatísticas da validação cruzada. Estas estatísticas nos dão uma confiança que, na média, este é o desempenho esperado
    #do classificador no mundo real.
    # print("min: %.2f, max: %.2f, avg +- std: %.2f+-%.2f" % (min(acuracias), max(acuracias), np.mean(acuracias), np.std(acuracias)))
    return acuracias

In [ ]:
def calcular_estatisticas(resultados):
    return np.mean(resultados), np.std(resultados), np.min(resultados), np.max(resultados)

def imprimir_estatisticas(resultados):
    media, desvio, mini, maxi = calcular_estatisticas(resultados)
    print("Resultados: %.2f +- %.2f, min: %.2f, max: %.2f" % (media, desvio, mini, maxi))

In [ ]:
accs_knn = do_knn()
accs_svm = do_svm()

In [ ]:
imprimir_estatisticas(accs_knn)
imprimir_estatisticas(accs_svm)

In [ ]:
from scipy.stats import ttest_ind_from_stats

In [ ]:
# calculamos a média e o desvio padrão dos resultados
media_knn, std_knn, _, _ = calcular_estatisticas(accs_knn)
media_svm, std_svm, _, _ = calcular_estatisticas(accs_svm)

#calcular o pvalor usando o teste t de Student para duas amostras independentes
_, pvalor = ttest_ind_from_stats(media_knn, std_knn, len(accs_knn), media_svm, std_svm, len(accs_svm))

In [ ]:
def rejeitar_hip_nula(media_amostral1, desvio_padrao_amostral1, n1, media_amostral2, desvio_padrao_amostral2, n2, alpha=0.05):
    _, pvalor = ttest_ind_from_stats(media_amostral1, desvio_padrao_amostral1, n1, media_amostral2, desvio_padrao_amostral2, n2)
    return pvalor <= alpha

In [ ]:
rejeitar_hip_nula(media_knn, std_knn, len(accs_knn), media_svm, std_svm, len(accs_svm))